# Sources
* Model architecture based on [Li et al., 2020](https://academic.oup.com/bioinformatics/article/36/4/1057/5578482?login=true), Repository available via download section on [Homepage](https://deepcleave.erc.monash.edu/)
* JoCoR loss function and training process adaptations are based on [Wei et al., 2020](https://openaccess.thecvf.com/content_CVPR_2020/html/Wei_Combating_Noisy_Labels_by_Agreement_A_Joint_Training_Method_with_CVPR_2020_paper.html), and official implementation on [Github](https://github.com/hongxin001/JoCoR)


In [1]:
import os
import sys
import csv
import random
import math
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, "r") as csvfile:
        train_data = list(csv.reader(csvfile))[1:]  # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls


def apply_random_masking(seq, num_tokens):
    """
    Mask `num_tokens` as 0 at random positions per sequence.
    """
    dist = torch.rand(seq.shape)
    m, _ = torch.topk(dist, num_tokens)
    return seq * (dist < m)


def regularized_auc(train_auc, dev_auc, threshold=0.0025):
    """
    Returns development AUC if overfitting is below threshold, otherwise 0.
    """
    return dev_auc if (train_auc - dev_auc) < threshold else 0


def gelu(x):
    """
    Facebook Research implementation of the gelu activation function.
    
    For information: OpenAI GPT's gelu is slightly different
    (and gives slightly different results):
    0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


def trainable_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def total_model_params(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
class TrainBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.float)
        self.seq = apply_random_masking(seq, num_tokens=1)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def train_wrapper(batch):
    return TrainBatch(batch)


class EvalBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.float)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def eval_wrapper(batch):
    return EvalBatch(batch)

In [5]:
class Attention(nn.Module):
    def __init__(self, input_dim, hidden):
        super().__init__()

        self.W0 = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(input_dim, hidden)).to(device)
        )
        self.W = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(hidden, 1)).to(device)
        )
        self.b0 = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(hidden, 1)).squeeze().to(device)
        )
        self.b = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(1, 1)).squeeze(1).to(device)
        )

    def forward(self, x):
        # input shape: (batch_size, num_filters, seq_len)
        # input needs to be permuted
        energy = (
            x.permute(0, 2, 1) @ self.W0 + self.b0
        )  # linear activation, i.e. only identity

        # input shape: (batch_size, seq_len, hidden)
        energy = (energy @ self.W + self.b).squeeze()

        # input shape: (batch_size, seq_len)
        energy = F.softmax(energy, dim=1)

        # input shape: energy=(batch_size, seq_len)
        # output shape: (batch_size, input_dim)
        # batch-wise dot product along dims 1
        res = (energy.unsqueeze(1) @ x.permute(0, 2, 1)).squeeze(1)

        # input shape: (batch_size, input_dim)
        # output shape: (batch_size, input_dim+seq_len)
        return torch.concat([res, energy], axis=-1)

In [6]:
class CNNAttention(nn.Module):
    def __init__(
        self,
        seq_len,
        num_filters1,
        kernel_size1,
        num_filters2,
        kernel_size2a,
        kernel_size2b,
        kernel_size2c,
        num_filters3,
        kernel_size3a,
        kernel_size3b,
        kernel_size3c,
        attention_hidden1,
        attention_hidden2,
        hidden_size1,
        hidden_size2,
        dropout
    ):
        super().__init__()

        self.dropout = nn.Dropout(dropout)

        self.conv1 = nn.Conv1d(
            in_channels=1,
            out_channels=num_filters1,
            kernel_size=kernel_size1,
            padding="same",
        )

        self.conv2a = nn.Conv1d(
            in_channels=num_filters1,
            out_channels=num_filters2,
            kernel_size=kernel_size2a,
            padding="same",
        )

        self.conv2b = nn.Conv1d(
            in_channels=num_filters1,
            out_channels=num_filters2,
            kernel_size=kernel_size2b,
            padding="same",
        )

        self.conv2c = nn.Conv1d(
            in_channels=num_filters1,
            out_channels=num_filters2,
            kernel_size=kernel_size2c,
            padding="same",
        )

        self.conv3a = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,
            kernel_size=kernel_size3a,
            padding="same",
        )

        self.conv3b = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,
            kernel_size=kernel_size3b,
            padding="same",
        )

        self.conv3c = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,
            kernel_size=kernel_size3c,
            padding="same",
        )

        self.attention1 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention1r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.attention2 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention2r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.attention3 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention3r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.fc1 = nn.Linear(3*num_filters3 + 3*seq_len, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, 1)

    def forward(self, seq):
        ### encoder ###

        # in: (batch_size, seq_len), out: (batch_size, num_filters1, seq_len)
        x1 = F.leaky_relu(self.dropout(self.conv1(seq.unsqueeze(1))))  # add dim=1 for conv

        # in: (batch_size, num_filters1, seq_len)
        # out: (batch_size, num_filters2, seq_len)
        y1 = F.leaky_relu(self.dropout(self.conv2a(x1)))
        y2 = F.leaky_relu(self.dropout(self.conv2b(x1)))
        y3 = F.leaky_relu(self.dropout(self.conv2c(x1)))
        # cat_y out: (batch_size, 3*num_filters2, seq_len)
        cat_y = self.dropout(torch.cat([y1, y2, y3], dim=1))

        # in: (batch_size, 3*num_filters2, seq_len)
        # out: (batch_size, num_filters3, seq_len)
        z1 = self.dropout(F.leaky_relu(self.conv3a(cat_y)))
        z2 = self.dropout(F.leaky_relu(self.conv3b(cat_y)))
        z3 = self.dropout(F.leaky_relu(self.conv3c(cat_y)))

        ### decoder ###

        # in: (batch_size, num_filters3, seq_len)
        # out w/out r, e.g. decoded_z1: (batch_size, num_filters3)
        # out w/ r: e.g. decoded_z1r: (batch_size, seq_len)
        decoded_z1 = self.attention1(z1)[:, : z1.shape[1]]
        decoded_z1r = self.attention1r(z1.permute(0, 2, 1))[:, : z1.shape[2]]

        decoded_z2 = self.attention2(z2)[:, : z2.shape[1]]
        decoded_z2r = self.attention2r(z2.permute(0, 2, 1))[:, : z2.shape[2]]

        decoded_z3 = self.attention3(z3)[:, : z3.shape[1]]
        decoded_z3r = self.attention3r(z3.permute(0, 2, 1))[:, : z3.shape[2]]

        # out: (batch_size, 3*num_filters3 + 3*seq_len)
        cat = self.dropout(
            torch.cat(
                [
                    decoded_z1,
                    decoded_z1r,
                    decoded_z2,
                    decoded_z2r,
                    decoded_z3,
                    decoded_z3r,
                ],
                dim=-1,
            )
        )

        # in: (batch_size, 3*num_filters3 + 3*seq_len), out: (batch_size, hidden_size1)
        out = self.dropout(F.leaky_relu(self.fc1(cat)))
        # in: (batch_size, hidden_size1), out: (batch_size, hidden_size2)
        out = self.dropout(F.leaky_relu(self.fc2(out)))
        # in: (batch_size, hidden_size2), out: (batch_size, 1)
        return self.fc3(out).squeeze()

In [7]:
def kl_loss_compute(pred, soft_targets):
    # adjusted for binary case
    kl = F.kl_div(F.logsigmoid(pred), torch.sigmoid(soft_targets), reduction='none')
    return torch.sum(kl)


class JoCoRLoss:
    def __call__(self, y1, y2, lbls, forget_rate, loss_func, kl_loss, co_lambda=0.1):
        loss_pick_1 = loss_func(y1, lbls) * (1 - co_lambda)
        loss_pick_2 = loss_func(y2, lbls) * (1 - co_lambda)
        loss_pick = (
            loss_pick_1
            + loss_pick_2
            + co_lambda * kl_loss_compute(y1, y2)
            + co_lambda * kl_loss_compute(y2, y1)
        ).cpu()

        ind_sorted = np.argsort(loss_pick.data)
        loss_sorted = loss_pick[ind_sorted]

        remember_rate = 1 - forget_rate
        num_remember = int(remember_rate * len(loss_sorted))

        ind_update = ind_sorted[:num_remember]

        loss = torch.mean(loss_pick[ind_update])

        return loss, loss

In [8]:
def train(model1, model2, optim, loss_func, loader, forget_rate, BCEWLL, kl_loss):
    epoch_loss1, num_correct1, total = 0, 0, 0
    epoch_loss2, num_correct2 = 0, 0
    preds1, preds2, lbls = [], [], []
    
    for batch in tqdm(loader, desc="Train: ", file=sys.stdout, unit="batches"):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores1 = model1(seq)
        scores2 = model2(seq)

        # JoCoR loss
        loss1, loss2 = loss_func(scores1, scores2, lbl, forget_rate, BCEWLL, kl_loss)
        
        optim.zero_grad()
        loss1.backward()
        optim.step()
        
        epoch_loss1 += loss1.item()
        epoch_loss2 += loss2.item()
        num_correct1 += ((scores1 > 0) == lbl).sum().item()
        num_correct2 += ((scores2 > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds1.extend(scores1.detach().tolist())
        preds2.extend(scores2.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return (
        epoch_loss1 / total,
        epoch_loss2 / total,
        num_correct1 / total,
        num_correct2 / total,
        roc_auc_score(lbls, preds1),
        roc_auc_score(lbls, preds2)
    )

In [9]:
def evaluate(model1, model2, loader):
    num_correct1, num_correct2, total = 0, 0, 0
    preds1, preds2, lbls = [], [], []
    
    for batch in tqdm(loader, desc="Eval: ", file=sys.stdout, unit="batches"):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores1 = model1(seq)
        scores2 = model2(seq)
        
        num_correct1 += ((scores1 > 0) == lbl).sum().item()
        num_correct2 += ((scores2 > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds1.extend(scores1.detach().tolist())
        preds2.extend(scores2.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return (
        num_correct1 / total,
        num_correct2 / total,
        roc_auc_score(lbls, preds1),
        roc_auc_score(lbls, preds2)
    )

In [10]:
def test(model, loader):
    num_correct, total = 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(loader, desc="Eval: ", file=sys.stdout, unit="batches"):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq)
        
        num_correct += ((scores > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
        
    return num_correct / total, roc_auc_score(lbls, preds)

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load train and dev data
train_seqs, train_lbl = read_data('../../data/c_train.csv')
dev_seqs, dev_lbl = read_data('../../data/c_val.csv')

# create vocab from train seqs
vocab = build_vocab_from_iterator(train_seqs, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [12]:
NUM_EPOCHS = 60
BATCH_SIZE = 512
VOCAB_SIZE = len(vocab)
LEARNING_RATE = 3e-4

NUM_GRADUAL = 10 # how many epochs for linear drop rate
NOISY_RATE = 0.2 # assumed
FORGET_RATE = NOISY_RATE / 2 # recommendation when facing asymmetric loss
EXPONENT = 1

# define drop rate schedule
rate_schedule = np.ones(NUM_EPOCHS)*FORGET_RATE
rate_schedule[:NUM_GRADUAL] = np.linspace(0, FORGET_RATE**EXPONENT, NUM_GRADUAL)

params = {
    'seq_len': 10,
    'kernel_size1': 1,
    'kernel_size2a': 3,
    'kernel_size2b': 17,
    'kernel_size2c': 13,
    'kernel_size3a': 11,
    'kernel_size3b': 15,
    'kernel_size3c': 19,
    'num_filters1': 220,
    'num_filters2': 262,
    'num_filters3': 398,
    'attention_hidden1': 3,
    'attention_hidden2': 2,
    'hidden_size1': 79,
    'hidden_size2': 24,
    'dropout': 0.04,
}

model1 = CNNAttention(**params).to(device)
model2 = CNNAttention(**params).to(device)

BCEWLL = nn.BCEWithLogitsLoss(reduction='none')
criterion = JoCoRLoss()
optimizer = optim.Adam(list(model1.parameters()) + list(model2.parameters()), lr=LEARNING_RATE)

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=train_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size = BATCH_SIZE, shuffle=True, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

In [13]:
start = time()
print("Starting Training.")
highest_val_auc = 0

# normal training loop
for epoch in range(1, NUM_EPOCHS + 1):
    model1.train()
    model2.train()
    train_loss1, train_loss2, train_acc1, train_acc2, train_auc1, train_auc2 = train(
        model1=model1,
        model2=model2,
        optim=optimizer,
        loss_func=criterion, # JoCoRLoss
        loader=train_loader,
        forget_rate=rate_schedule[epoch-1],
        BCEWLL=BCEWLL, # nn.BCEWithLogitsLoss with reduction=none
        kl_loss=kl_loss_compute
    )

    model1.eval()
    model2.eval()
    with torch.no_grad():
        val_acc1, val_acc2, val_auc1, val_auc2 = evaluate(
            model1=model1,
            model2=model2,
            loader=dev_loader,
        )
    
    print(
        f"Model 1 Training:   [Epoch {epoch:2d}, Loss: {train_loss1:8.6f}, Acc: {train_acc1:.4f}, AUC: {train_auc1:.4f}]"
    )
    print(
        f"Model 1 Evaluation: [Epoch {epoch:2d}, Acc: {val_acc1:.4f}, AUC: {val_auc1:.4f}]"
    )
    print(
        f"Model 2 Training:   [Epoch {epoch:2d}, Loss: {train_loss2:8.6f}, Acc: {train_acc2:.4f}, AUC: {train_auc2:.4f}]"
    )
    print(
        f"Model 2 Evaluation: [Epoch {epoch:2d}, Acc: {val_acc2:.4f}, AUC: {val_auc2:.4f}]"
    )

    if val_auc1 > val_auc2:
        reg_auc = regularized_auc(train_auc1, val_auc1, threshold=0)
        model = model1.state_dict()
        print('saved model1')
    else:
        reg_auc = regularized_auc(train_auc2, val_auc2, threshold=0)
        model = model2.state_dict()
        print('saved model2')
    if reg_auc > highest_val_auc:
        highest_val_auc = reg_auc
        path = f"../../params/c_term/cnnAttention_jocor/auc{reg_auc:.4f}_epoch{epoch}.pt"
        torch.save(model, path)

print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

Starting Training.
Eval: 100%|███████████████████████████████████████████████████████| 278/278 [00:03<00:00, 73.68batches/s]
Model 1 Training:   [Epoch  1, Loss: 0.001895, Acc: 0.8228, AUC: 0.5383]
Model 1 Evaluation: [Epoch  1, Acc: 0.8215, AUC: 0.6238]
Model 2 Training:   [Epoch  1, Loss: 0.001895, Acc: 0.8231, AUC: 0.5369]
Model 2 Evaluation: [Epoch  1, Acc: 0.8215, AUC: 0.6211]
saved model1
Eval: 100%|███████████████████████████████████████████████████████| 278/278 [00:03<00:00, 75.34batches/s]
Model 1 Training:   [Epoch  2, Loss: 0.001687, Acc: 0.8231, AUC: 0.6394]
Model 1 Evaluation: [Epoch  2, Acc: 0.8215, AUC: 0.6824]
Model 2 Training:   [Epoch  2, Loss: 0.001687, Acc: 0.8231, AUC: 0.6380]
Model 2 Evaluation: [Epoch  2, Acc: 0.8215, AUC: 0.6822]
saved model1
Eval: 100%|███████████████████████████████████████████████████████| 278/278 [00:03<00:00, 74.82batches/s]
Model 1 Training:   [Epoch  3, Loss: 0.001531, Acc: 0.8231, AUC: 0.6705]
Model 1 Evaluation: [Epoch  3, Acc: 0.8215, 

In [14]:
test_path = '../../data/c_test.csv'
test_seqs, test_lbls = read_data(test_path)

test_data = CleavageDataset(test_seqs, test_lbls)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

# load best model, evaluate on test set
best_model = sorted(
    [f for f in os.listdir("../../params/c_term/cnnAttention_jocor/") if f.endswith(".pt")],
    reverse=True,
)[0]
print("Loaded model: ", best_model)
model1.load_state_dict(torch.load('../../params/c_term/cnnAttention_jocor/' + best_model))
model1.eval()
with torch.no_grad():
    test_acc, test_auc = test(model1, test_loader)
print(
    f"Test Set Performance: Acc: {test_acc:.4f}, AUC: {test_auc:.4f}"
)
print(
    f"Total model params: {total_model_params(model1)}, trainable model params: {trainable_model_params(model1)}"
)

Loaded model:  auc0.7144_epoch59.pt
Eval: 100%|██████████████████████████████████████████████████████| 278/278 [00:01<00:00, 139.04batches/s]
Test Set Performance: Acc: 0.8292, AUC: 0.7110
Total model params: 16084198, trainable model params: 16084198
